**Use SQL kernel to run code in this notebook**  

This notebook provides code that creates Azure SQL Database objects, required for Logic App integration.

Assumtion is that Azure SQL Database already exists.

First we'll create Azure SQL Database schema. AzureConsumption can be replaced with any other valid schema name, in which case, code that follows will need to be changed to reflect different schema name.

In [ ]:
CREATE SCHEMA AzureConsumption;

Now we can create Usage table where Azure usage data will be stored. Data will be retreived using Azure Consumption Usage Details REST API - [Usage Details - List - REST API (Azure Consumption) | Microsoft Docs](https://docs.microsoft.com/en-us/rest/api/consumption/usage-details/list)

Note: Last three "tags" columns assume that Azure resources are tagged with Environment, Owner and Cost Center tags. These tags can be removed or changed to fit specific implementation.

In [ ]:
CREATE TABLE [AzureConsumption].[Usage]
(
    subscriptionId char(512),
    datetime char(512),
    quantity float,
    effectivePrice float,
    cost float,
    billingCurrency char(512),
    resourceId char(512),
    resourceName char(512),
    resourceGroup char(512),
    consumedService char(512),
    tagsEnvironment char(512),
    tagsOwner char(512),
    tagsCostCenter char(512)
)
;

Next is stored procedure that will upsert (update or insert) records

In [ ]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE PROCEDURE [AzureConsumption].[sp_UpsertUsage]
(
    @subscriptionId char(512),
    @datetime char(512),
    @quantity float,
    @effectivePrice float,
    @cost float,
    @billingCurrency char(512),
    @resourceId char(512),
    @resourceName char(512),
    @resourceGroup char(512),
    @consumedService char(512),
    @tagsEnvironment char(512),
    @tagsOwner char(512),
    @tagsCostCenter char(512)
)
AS
BEGIN
    IF EXISTS (SELECT 1 FROM [AzureConsumption].[Usage]
    WHERE subscriptionId = @subscriptionId
    AND datetime = @datetime
    AND resourceId = @resourceId
    AND resourceName = @resourceName
    AND resourceGroup = @resourceGroup
    AND consumedService = @consumedService
    AND quantity = @quantity
    AND effectivePrice = @effectivePrice
    AND cost = @cost
    AND billingCurrency = @billingCurrency
	)
    BEGIN
        UPDATE [AzureConsumption].[Usage] SET 
            tagsEnvironment = @tagsEnvironment,
            tagsOwner = @tagsOwner,
            tagsCostCenter = @tagsCostCenter
        WHERE subscriptionId = @subscriptionId
        AND datetime = @datetime
        AND resourceId = @resourceId
        AND resourceGroup = @resourceGroup
	PRINT 'Usage Data Updated'
    END
ELSE
    BEGIN
        INSERT INTO [AzureConsumption].[Usage] VALUES (@subscriptionId, @datetime, @quantity, @effectivePrice, @cost, @billingCurrency, @resourceId, @resourceName, @resourceGroup, @consumedService, @tagsEnvironment, @tagsOwner, @tagsCostCenter)
	PRINT 'New Usage Data Inserted'
    END
END
GO


At this point, stored procedure can be tested with sample data ...

In [ ]:
EXEC [AzureConsumption].[sp_UpsertUsage] 'F89D5FA7-353B-4486-8700-99FC72E6AAAA','2021-11-01T00:00:00.0000000Z','0.333336','0.059046','0.019682157456','CHF','/subscriptions/f89d5fa7-353b-4486-8700-99fc72e6aaaa/resourceGroups/platform/providers/Microsoft.Storage/storageAccounts/teststorageaccount','teststorageaccount','platform','Microsoft.Storage','Dev','',''

To allow Logic App to access relevant Azure SQL Database objects (e.g. table, stored procedure), new user and role need to be created.

In example below, _logic\_app\_azure\_consumption_ is a name of Logic App.

Note: db\_datawriter and db\_datareader roles are not enough because Logic App can't see stored procedure. db\_owner works but possibly some other less priviledged role might work.

In [ ]:
DROP USER IF EXISTS logic_app_azure_consumption /* Remove user, in case user with same name already exists */

In [ ]:
DECLARE @LogicAppName VARCHAR(50)
DECLARE @cmd VARCHAR(200)

/* Change LogicAppName value to reflect provisioned Logic App name */
SET @LogicAppName = N'<Logic App name>'

/* Remove user, in case user with same name already exists */
SET @cmd = 'DROP USER IF EXISTS ' + @LogicAppName
EXEC(@cmd)


SET @cmd = 'CREATE USER ' + @LogicAppName + ' FROM EXTERNAL PROVIDER'
EXEC(@cmd)

SET @cmd = 'ALTER ROLE db_owner ADD MEMBER ' + @LogicAppName
EXEC(@cmd)

Below is some utility code that can help with testing ...

In [ ]:
DROP TABLE [AzureConsumption].[Usage] /* In case Usage table needs to be deleted */

In [ ]:
DROP PROCEDURE [AzureConsumption].[sp_UpsertUsage] /* In case stored procedure needs to be deleted */

In [ ]:
SELECT TOP (1000) * FROM [AzureConsumption].[Usage] ORDER BY datetime /* Is there anything in the table? */

In [ ]:
DELETE from [AzureConsumption].[Usage] /* In case records from Usage table need to be deleted */